# OME-Zarr Image Exploration

In this notebook we will show how to use the 'NgffImage' class to explore and manage an OME-NGFF image.

For this example we will use a small example image that can be downloaded from the following link: [example ome-zarr](https://zenodo.org/records/13305156)

## Setup

You can download the example image (on Linux and Mac os) by running the following command:

```bash
bash setup_data.sh
```
from the root of the repository.

## NgffImage

The `NgffImage` provides a high-level interface to read, write and manipulate NGFF images.
A `NgffImage` can be created from a storelike object (e.g. a path to a directory, or a url) or from a `zarr.Group` object. 

In [ ]:
from ngio.core import NgffImage

# Ngio can stream data from any fsspec-compatible store
path = "../../data/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0"
ngff_image = NgffImage(path, "r")


The `ngff_image` object provides a high-level interface to read, write and manipulate OME-Zarr images.

Print the image will show some overview information like:
* The path to the image
* The multiscale pyramid paths
* The labels contained in the image
* The tables contained in the imag

In [ ]:
print(ngff_image)

In [ ]:
From the NgffImage object, you can access the image data as a numpy ar

From the `NgffImage` object we can easily access access the image data (at any resolution level), the labels and the tables.

Get a single `level` of the image pyramid as `Image` (to know more about the `Image` class, please refer to the [Image notebook](./image.ipynb)
The `Image` object is the main object to interact with the image. 
It contains methods to interact with the image data and metadata.

In [ ]:
from ngio.ngff_meta import PixelSize

# 1. Get image from highest resolution (default)
image = ngff_image.get_image()
print(image)

# 2. Get image from a specific level using the path keyword
image = ngff_image.get_image(path="1")
print(image)

# 3. Get image from a specific pixel size using the pixel_size keyword
image = ngff_image.get_image(pixel_size=PixelSize(x=0.65, y=0.65, z=1))
print(image)

The `Image` object provides a high-level interface to read and write image data at a specific resolution level.

In [ ]:
print("Shape", image.shape)
print("Axes", image.axes_names)
print("PixelSize", image.pixel_size)
print("Dimensions", image.dimensions)
print("Channel Names", image.channel_labels)

In [ ]:
# Get data as a numpy array or a dask array
data = image.get_array(c=0, mode="numpy")
print(data)

dask_data = image.get_array(c=0, mode="dask")
dask_data

`ngio` design is to always provide the data in a canonical axis order (`t`, `c`, `z`, `y`, `x`) no matter what is the order on disk.  
The `Image` object provides methods to access the data in this order.  
If you want to access data or metadata in the on-disk order, you can by using `on_disk_{method_name}` methods.


In [ ]:
print("On-disk shape", image.on_disk_shape)
print("On-disk array", image.on_disk_array)
print("On-disk dask array", image.on_disk_dask_array)

## Labels

The `NgffImage` can also be used to load labels from a `OME-NGFF` file and behave similarly to the `Image` object.

In [ ]:
print("List of Labels: ", ngff_image.labels.list())

label_nuclei = ngff_image.labels.get_label("nuclei", path="0")
print(label_nuclei)

## Tables

The `NgffImage` can also be used to load tables from a `OME-NGFF` file.

`ngio` supports three types of tables:
 - `features table` A simple table to store features associated with a label.
 - `roi table` A table to store regions of interest.
 - `masking roi tables` A table to store single objects bounding boxes associated with a label.

In [ ]:
print("List of Tables: ", ngff_image.tables.list())
print(" - Feature tables: ", ngff_image.tables.list(table_type='feature_table'))
print(" - Roi tables: ", ngff_image.tables.list(table_type='roi_table'))
print(" - Masking Roi tables: ", ngff_image.tables.list(table_type='masking_roi_table'))

In [ ]:
# Loading a table
feature_table = ngff_image.tables.get_table("regionprops_DAPI")
feature_table.table

In [ ]:
# Loading a roi table
roi_table = ngff_image.tables.get_table("FOV_ROI_table")

print(f"{roi_table.field_indexes=}")
print(f"{roi_table.get_roi('FOV_1')=}")

roi_table.table

Rois can be used to index image and label data.

In [ ]:
import matplotlib.pyplot as plt

# Plotting a single ROI
roi = roi_table.get_roi("FOV_1")
roi_data = image.get_array_from_roi(roi, c=0, mode="numpy")
plt.title("ROI: FOV_1")
plt.imshow(roi_data[0], cmap="gray")
plt.axis("off")
plt.show()

## Derive a new NgffImage

When processing an image, it is often useful to derive a new image from the original image.
The `NgffImage` class provides a method to derive a new image from the original image.
When deriving a new image, a new `NgffImage` object is created with the same metadata as the original image. Optionally the 
user can specify different metadata for the new image(.e.g. different channels names).

In [ ]:
new_ngff_image = ngff_image.derive_new_image("../../data/new_ome.zarr", name="new_image")
print(new_ngff_image)

## Steam an NgffImage over HTTP

The `NgffImage` class can also be used to stream an image over HTTP. This is useful when the image is stored on a remote server and you want to access it without downloading the entire image. All features of the `NgffImage` class are available when streaming an image over HTTP (besides anything that requires writing to the image).

In [ ]:
from ngio.core.utils import get_fsspec_http_store

# Ngio can stream data from any fsspec-compatible store
url = "https://raw.githubusercontent.com/fractal-analytics-platform/fractal-ome-zarr-examples/refs/heads/main/v04/20200812-CardiomyocyteDifferentiation14-Cycle1_B_03_mip.zarr/"
store = get_fsspec_http_store(url)
ngff_image = NgffImage(store, "r")

print(ngff_image)